In [1]:
import re
import os
import csv
import time
import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome()
url = 'https://new.land.naver.com/complexes/111515?ms=37.49751,127.1042975,17&a=APT:ABYG:JGC&e=RETAIL'  #헬리오시티
browser.get(url)


In [107]:
naver_price_lines = browser.find_elements_by_class_name('price_line')
naver_price_line = []
for naver_price_line_tmp in naver_price_lines:
    a = naver_price_line_tmp.text
    naver_price_line.append(a)

naver_info_areas = browser.find_elements_by_class_name('info_area')
naver_info_area = []
for naver_info_area_tmp in naver_info_areas:
    b = naver_info_area_tmp.text.split()
    naver_info_area.append(b)

df = pd.DataFrame(
    {'A': naver_price_line,
     'B': naver_info_area
    })

In [109]:
# 전월세 구분해서 div에 새로 칼럼 추가
def price_cut(row):
   row = row[:2]
   return row
df['div'] = df['A'].map(price_cut)

# 가격
def price_cal(row):
    row = row[2:]
    if len(row) <= 6:
        row = row.replace('억', '0000')
    else:
        row = row.replace('억 ', '').replace(',', '')
    # row = row + "/0"
    return row

df['price'] = df['A'].map(price_cal)

# def price_cal2(row):
#     row = row[2:]
#     if len(row) < 9:
#         row = row.replace('억', '0000')
#     else:
#         row = row.replace('억 ', '').replace(',', '')
#     return row

# for r in df['div']:
#     if r == "매매":
#         df['price'] = df['A'].map(price_cal)
#     elif r == "전세":
#         df['price'] = df['A'].map(price_cal)
#     else:
#         df['price'] = df['A'].map(price_cal2)

def price_cut1(row):
    row = row.split('/')
    if len(row) == 2:
        row = row[1]
    else:
        row = 0
    return row

df['price1'] = df['price'].map(price_cut1)

def price_cut2(row):
    return row.split('/')[0]
df['price'] = df['price'].apply(price_cut2)

df

,A,B,div,price,price1
0,월세3억/270,"[아파트107J1/84m²,, 4/15층,, 남향, 33p., BEST추천, 아침전...",월세,30000,270
1,월세5억/195,"[아파트107A/84m²,, 10/24층,, 남향, 33p, 역세권, 4BAY, 인...",월세,50000,195
2,월세1억/470,"[아파트141B/110m²,, 11/35층,, 남향, 42p, 확트인뷰, 빠른입주가...",월세,10000,470
3,"전세9억 5,000","[아파트84/59m²,, 10/12층,, 남향, 25p, 탁트인, 전망.밝은, 남향...",전세,95000,0
4,월세1억/250,"[아파트84/59m²,, 6/12층,, 남향, 25p, 남향, 빠른입주가능, 송파역...",월세,10000,250
5,전세7억,"[아파트61A/39m²,, 15/21층,, 남서향, 18p, 풀에어컨, 채광통풍전망...",전세,70000,0
6,"매매25억 5,000","[아파트140A/110m²,, 7/35층,, 남서향, 42p, 확트인, 탄천조망, ...",매매,255000,0
7,월세8억/230,"[아파트126/99m²,, 29/35층,, 남향, 38,, 선택적, 자연경관뷰, ,...",월세,80000,230
8,"매매19억 8,000","[아파트110D/84m²,, 19/35층,, 남동향, 33P, 입주및, 갭투자가능,...",매매,198000,0
9,"전세9억 5,000","[아파트84/59m²,, 10/22층,, 남서향, 25p완전귀한평형,시원한, 통풍,...",전세,95000,0


In [110]:
def area1(row):
    row = row[0:1]
    row = "".join(row)
    row = row.replace('아파트', '').replace(',', '')
    row = row.split('/')[0]
    return row

df['area1'] = df['B'].map(area1)

def area1_1(row):
    row = row[0:1]
    row = "".join(row)
    row = row.replace('아파트', '').replace(',', '').replace('m²', '')
    row = row.split('/')[1]
    return row

df['area1_1'] = df['B'].map(area1_1)

# area2
def area2(row):
    row = row[1:2]
    row = "".join(row)
    row = row.replace('층,', '')
    row = row.split('/')[0]
    return row

df['area2'] = df['B'].map(area2)

def area2_1(row):
    row = row[1:2]
    row = "".join(row)
    row = row.replace('층,', '')
    row = row.split('/')[1]
    return row

df['area2_1'] = df['B'].map(area2_1)

def area3(row):
    row = row[2:3]
    row = "".join(row)
    return row

df['area3'] = df['B'].map(area3)
df

,A,B,div,price,price1,area1,area1_1,area2,area2_1,area3
0,월세3억/270,"[아파트107J1/84m²,, 4/15층,, 남향, 33p., BEST추천, 아침전...",월세,30000,270,107J1,84,4,15,남향
1,월세5억/195,"[아파트107A/84m²,, 10/24층,, 남향, 33p, 역세권, 4BAY, 인...",월세,50000,195,107A,84,10,24,남향
2,월세1억/470,"[아파트141B/110m²,, 11/35층,, 남향, 42p, 확트인뷰, 빠른입주가...",월세,10000,470,141B,110,11,35,남향
3,"전세9억 5,000","[아파트84/59m²,, 10/12층,, 남향, 25p, 탁트인, 전망.밝은, 남향...",전세,95000,0,84,59,10,12,남향
4,월세1억/250,"[아파트84/59m²,, 6/12층,, 남향, 25p, 남향, 빠른입주가능, 송파역...",월세,10000,250,84,59,6,12,남향
5,전세7억,"[아파트61A/39m²,, 15/21층,, 남서향, 18p, 풀에어컨, 채광통풍전망...",전세,70000,0,61A,39,15,21,남서향
6,"매매25억 5,000","[아파트140A/110m²,, 7/35층,, 남서향, 42p, 확트인, 탄천조망, ...",매매,255000,0,140A,110,7,35,남서향
7,월세8억/230,"[아파트126/99m²,, 29/35층,, 남향, 38,, 선택적, 자연경관뷰, ,...",월세,80000,230,126,99,29,35,남향
8,"매매19억 8,000","[아파트110D/84m²,, 19/35층,, 남동향, 33P, 입주및, 갭투자가능,...",매매,198000,0,110D,84,19,35,남동향
9,"전세9억 5,000","[아파트84/59m²,, 10/22층,, 남서향, 25p완전귀한평형,시원한, 통풍,...",전세,95000,0,84,59,10,22,남서향


In [141]:
df = df.astype({'area1_1': 'int'})
df.sort_values(by=['area1_1'], axis=0, ascending=False, inplace=True)
df

,A,B,div,price,price1,area1,area1_1,area2,area2_1,area3
59,"매매38억 5,000","[아파트193B/150m²,, 35/35층,, 남향, 60p, 펜트하우스, 대모산,...",매매,385000,0,193B,150,35,35,남향
39,"매매38억 5,000","[아파트193B/150m²,, 35/35층,, 남향, 60p, 팬트하우스, 강남대모...",매매,385000,0,193B,150,35,35,남향
30,"매매38억 5,000","[아파트193B/150m²,, 35/35층,, 남향, 60p, 전속, 펜트.대모산,...",매매,385000,0,193B,150,35,35,남향
2,월세1억/470,"[아파트141B/110m²,, 11/35층,, 남향, 42p, 확트인뷰, 빠른입주가...",월세,10000,470,141B,110,11,35,남향
6,"매매25억 5,000","[아파트140A/110m²,, 7/35층,, 남서향, 42p, 확트인, 탄천조망, ...",매매,255000,0,140A,110,7,35,남서향
42,월세1억/470,"[아파트141B/110m²,, 11/35층,, 남향, 42p,에어컨풀,빠른입주가능,...",월세,10000,470,141B,110,11,35,남향
52,"매매25억 5,000","[아파트140A/110m²,, 7/35층,, 서향, 42p입주협의, 커뮤니티, 골프...",매매,255000,0,140A,110,7,35,서향
7,월세8억/230,"[아파트126/99m²,, 29/35층,, 남향, 38,, 선택적, 자연경관뷰, ,...",월세,80000,230,126,99,29,35,남향
11,매매25억,"[아파트126/99m²,, 22/29층,, 남향, 38p, 귀한, 입주.갭투자, 가...",매매,250000,0,126,99,22,29,남향
57,전세17억,"[아파트126/99m²,, 23/29층,, 남향, 38P, 남향, 로얄동, 파크밴드...",전세,170000,0,126,99,23,29,남향


In [145]:
df1 = df[(df['div'] == '전세') & (df['area1_1'] == 84)]
#df[df['div'] == '월세']
#df1 
#df2 = df1['price'].mean()
#print(int(df2))
df1 = df1.astype({'price': 'int'})
print(df1.dtypes)
df1

A          object
B          object
div        object
price       int32
price1     object
area1      object
area1_1     int32
area2      object
area2_1    object
area3      object
dtype: object


,A,B,div,price,price1,area1,area1_1,area2,area2_1,area3
18,"전세11억 8,000","[아파트107A/84m²,, 7/27층,, 남향, 33p전속물건,판상형A타입,남향,...",전세,118000,0,107A,84,7,27,남향
19,전세12억,"[아파트107A/84m²,, 10/24층,, 남향, 전세, 33p, 남향, 초역세권...",전세,120000,0,107A,84,10,24,남향
40,"전세12억 5,000","[아파트109E/84m²,, 9/35층,, 남동향, 33pii4년계약가능6년거주가능...",전세,125000,0,109E,84,9,35,남동향


In [146]:
df2 = df1['price'].mean()
print(int(df2))

121000


In [111]:
df.to_html("test.html")